<a href="https://colab.research.google.com/github/himashwetha8/Prototype-to-Production-Building-Multi-Agent-Systems-Using-A2A-Protocol/blob/main/Multi_Agent_A2A_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#README
# Prototype to Production: Building Multi-Agent Systems Using A2A Protocol

#A compact multi-agent system demo that demonstrates agent-to-agent (A2A) communication, orchestration, tool usage (PDF/chart generation), and a production-style workflow using Python.

## What is included
#- `src/` : Python source with orchestrator, router, and agent modules
#- `notebooks/` : runnable Colab notebook demo (prototype_to_production_full.ipynb)
#- `docs/` : architecture diagram (640x360)
#- `examples/` : sample input and sample outputs

## Quickstart (local)
#1. Clone the repo.
#2. Create and activate a virtual environment:
   #```bash
   #python -m venv venv
   #source venv/bin/activate   # Linux / macOS
   #venv\Scripts\activate      # Windows


In [ ]:
# router.py
# A2A Router module
# Handles agent-to-agent message routing and keeps a communication log.

import datetime

class Router:
    """
    The Router handles all communication between agents.
    Acts like a message bus in multi-agent systems.
    """

    def __init__(self):
        self.log = []  # Stores all messages exchanged

    def send(self, sender, receiver, message):
        """
        Send a message from `sender` to `receiver`.
        `sender` and `receiver` are strings (agent names) or objects with .name attribute.
        Returns a human-readable entry string.
        """
        timestamp = datetime.datetime.now().strftime("%H:%M:%S")

        # Normalize names if objects provided
        from_name = sender.name if hasattr(sender, "name") else str(sender)
        to_name = receiver.name if hasattr(receiver, "name") else str(receiver)

        entry = {
            "time": timestamp,
            "from": from_name,
            "to": to_name,
            "message": message
        }

        self.log.append(entry)
        return f"[{timestamp}] {from_name} → {to_name}: {message}"

    def show_log(self):
        """
        Pretty print the A2A communication log.
        """
        if not self.log:
            print("Message log is empty.")
            return

        print("\n================ A2A COMMUNICATION LOG ================\n")
        for entry in self.log:
            print(f"[{entry['time']}] {entry['from']} → {entry['to']}: {entry['message']}")
        print("\n=========================================================\n")


# When run as a script, show a small sanity check
if __name__ == "__main__":
    r = Router()
    print("Router initialized.")
    if len(r.log) == 0:
        print("Message log is empty.")
    else:
        r.show_log()


Router initialized.
Message log is empty.


In [ ]:
# agent_base.py
# Base Agent class for the multi-agent system

class Agent:
    """
    Base class for all agents.
    Each agent can send + receive messages through the Router.
    """

    def __init__(self, name, router):
        self.name = name
        self.router = router

    def send(self, receiver, message):
        """
        Sends a message to another agent through the router.
        """
        return self.router.send(self, receiver, message)

    def receive(self, message, sender):
        """
        What happens when an agent receives a message.
        You can override this method in specialized agents.
        """
        return f"{self.name} received '{message}' from {sender}"


# Sanity test when running directly
if __name__ == "__main__":
    # The Router class should already be defined in the global scope from an earlier cell.
    # No need to import it like a module.

    router = Router()
    a1 = Agent("AgentA", router)
    a2 = Agent("AgentB", router)

    print(a1.send(a2, "Hello from A"))
    print(a2.send(a1, "Hello from B"))

    print("\nRouter Log:")
    router.show_log()

[06:33:59] AgentA → AgentB: Hello from A
[06:33:59] AgentB → AgentA: Hello from B

Router Log:

================ A2A COMMUNICATION LOG ================

[06:33:59] AgentA → AgentB: Hello from A
[06:33:59] AgentB → AgentA: Hello from B




In [ ]:
# specialized_agents.py
# Contains specialized agent classes for planning, research, analysis, and writing.

# from agent_base import Agent # Removed this line

# ---------------------------------------------------------------
# PLANNER AGENT
# ---------------------------------------------------------------

class Planner(Agent):
    """
    Decides the sequence of tasks and assigns work to other agents.
    """

    def plan(self, topic):
        plan = [
            f"Research information about: {topic}",
            "Analyze the collected data",
            "Generate a final report"
        ]
        return plan


# ---------------------------------------------------------------
# RESEARCHER AGENT
# ---------------------------------------------------------------

class Researcher(Agent):
    """
    Simulates data collection (in real systems this agent would call APIs).
    """

    def run(self, topic):
        collected = f"Raw Data → AI adoption increased 22% in 2024. Topic: {topic}"
        return collected


# ---------------------------------------------------------------
# ANALYST AGENT
# ---------------------------------------------------------------

class Analyst(Agent):
    """
    Simulates analysis of collected data.
    """

    def run(self, raw_data):
        analysis = f"Analysis → Trend detected based on data: ({raw_data})"
        return analysis


# ---------------------------------------------------------------
# WRITER AGENT
# ---------------------------------------------------------------

class Writer(Agent):
    """
    Generates a human-readable report.
    """

    def run(self, analysis):
        report = f"""
===============================
        FINAL REPORT
===============================

{analysis}

Generated automatically by Multi-Agent A2A System.
"""
        return report


# ---------------------------------------------------------------
# LOCAL SANITY TEST (optional)
# ---------------------------------------------------------------

if __name__ == "__main__":
    # from router import Router # Removed this line

    router = Router()

    planner = Planner("Planner", router)
    researcher = Researcher("Researcher", router)
    analyst = Analyst("Analyst", router)
    writer = Writer("Writer", router)

    topic = "AI Adoption"

    # Planner → Researcher
    plan = planner.plan(topic)
    print("Plan:", plan)
    planner.send(researcher, "Start researching now.")

    # Researcher → Analyst
    raw = researcher.run(topic)
    researcher.send(analyst, "Sending raw data.")

    # Analyst → Writer
    analysis = analyst.run(raw)
    analyst.send(writer, "Sending analysis.")

    # Writer → Output
    report = writer.run(analysis)
    print(report)

    print("\nRouter Log:")
    router.show_log()

Plan: ['Research information about: AI Adoption', 'Analyze the collected data', 'Generate a final report']

        FINAL REPORT

Analysis → Trend detected based on data: (Raw Data → AI adoption increased 22% in 2024. Topic: AI Adoption)

Generated automatically by Multi-Agent A2A System.


Router Log:

================ A2A COMMUNICATION LOG ================

[06:36:20] Planner → Researcher: Start researching now.
[06:36:20] Researcher → Analyst: Sending raw data.
[06:36:20] Analyst → Writer: Sending analysis.




In [ ]:
# tool_agent.py
# Tool agent for saving files and generating output artifacts.

# from agent_base import Agent # This line was removed/commented out

class ToolAgent(Agent):
    """
    Helps save files, generate output artifacts.
    """

    def save_text(self, text, filename):
        """
        Saves a text file to the given filename.
        Returns status message.
        """
        with open(filename, "w") as f:
            f.write(text)

        return f"File saved: {filename}"


# ---------------------------------------------------------------
# LOCAL SANITY TEST
# ---------------------------------------------------------------

if __name__ == "__main__":
    # from router import Router # Removed this line

    router = Router()
    tool = ToolAgent("ToolAgent", router)

    # Create a sample text
    sample_text = "This is a test file generated by ToolAgent."
    filename = "sample_output.txt"

    # Save the file
    result = tool.save_text(sample_text, filename)
    print(result)

    # Log message (just an example)
    tool.send(tool, f"Saved file: {filename}")

    print("\nRouter Log:")
    router.show_log()

File saved: sample_output.txt

Router Log:

================ A2A COMMUNICATION LOG ================

[06:37:50] ToolAgent → ToolAgent: Saved file: sample_output.txt




In [ ]:
# orchestrate.py
# Full pipeline orchestration for the multi-agent A2A system.

# from router import Router # Commented out
# from specialized_agents import Planner, Researcher, Analyst, Writer # Commented out
# from tool_agent import ToolAgent # Commented out

def orchestrate(topic="AI Adoption"):
    """
    Full end-to-end pipeline using the A2A protocol.
    Planner → Researcher → Analyst → Writer → ToolAgent
    """

    # Initialize router + all agents
    router = Router()
    planner = Planner("Planner", router)
    researcher = Researcher("Researcher", router)
    analyst = Analyst("Analyst", router)
    writer = Writer("Writer", router)
    tool = ToolAgent("ToolAgent", router)

    # --- Step 1: Planner creates a task plan ---
    plan = planner.plan(topic)
    planner.send(researcher, f"Start research on: {topic}")

    # --- Step 2: Researcher collects raw data ---
    raw_data = researcher.run(topic)
    researcher.send(analyst, "Raw data ready for analysis.")

    # --- Step 3: Analyst processes data ---
    analysis = analyst.run(raw_data)
    analyst.send(writer, "Analysis ready for reporting.")

    # --- Step 4: Writer generates final report ---
    report = writer.run(analysis)
    writer.send(tool, "Final report ready to save.")

    # --- Step 5: ToolAgent saves the final report ---
    filename = "A2A_Final_Report.txt"
    tool.save_text(report, filename)

    return router, report, filename


# ---------------------------------------------------------------
# LOCAL SANITY TEST
# ---------------------------------------------------------------
if __name__ == "__main__":
    router, final_report, saved_file = orchestrate("AI Adoption in 2024")

    print("============== FINAL REPORT ==============")
    print(final_report)
    print("========================================= \n")

    print("Saved File:", saved_file)

    print("\nRouter Log:")
    router.show_log()

============== FINAL REPORT ==============

        FINAL REPORT

Analysis → Trend detected based on data: (Raw Data → AI adoption increased 22% in 2024. Topic: AI Adoption in 2024)

Generated automatically by Multi-Agent A2A System.


Saved File: A2A_Final_Report.txt

Router Log:

================ A2A COMMUNICATION LOG ================

[06:39:07] Planner → Researcher: Start research on: AI Adoption in 2024
[06:39:07] Researcher → Analyst: Raw data ready for analysis.
[06:39:07] Analyst → Writer: Analysis ready for reporting.
[06:39:07] Writer → ToolAgent: Final report ready to save.




In [ ]:
# run_system.py
# Master runner for the full multi-agent A2A system.

# from orchestrate import orchestrate # Removed this line

if __name__ == "__main__":
    topic = "AI Adoption and Multi-Agent Collaboration"

    # Run full pipeline
    router, report, filename = orchestrate(topic)

    print("\n==================== FINAL REPORT ====================")
    print(report)
    print("======================================================\n")

    print(f"Saved Output File: {filename}\n")

    print("==================== A2A LOG ====================")
    router.show_log()


==================== FINAL REPORT ====================

        FINAL REPORT

Analysis → Trend detected based on data: (Raw Data → AI adoption increased 22% in 2024. Topic: AI Adoption and Multi-Agent Collaboration)

Generated automatically by Multi-Agent A2A System.


Saved Output File: A2A_Final_Report.txt

==================== A2A LOG ====================

================ A2A COMMUNICATION LOG ================

[06:40:13] Planner → Researcher: Start research on: AI Adoption and Multi-Agent Collaboration
[06:40:13] Researcher → Analyst: Raw data ready for analysis.
[06:40:13] Analyst → Writer: Analysis ready for reporting.
[06:40:13] Writer → ToolAgent: Final report ready to save.


